In [14]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [15]:
import pandas as pd
from pandas import DataFrame

# Yalnızca bir kere patch et
if not hasattr(DataFrame, "_orig_repr_html_"):
    DataFrame._orig_repr_html_ = DataFrame._repr_html_

    def _scrollable_repr_html(self):
        html = DataFrame._orig_repr_html_(self)  # sınıftaki orijinale sabitle
        return (
            '<div style="width:100%; height:400px; overflow:auto; '
            'border:1px solid #eee; border-radius:8px; padding:8px;">'
            f'{html}</div>'
        )

    DataFrame._repr_html_ = _scrollable_repr_html

### Sipariş verisi

In [16]:
canter_sipariş=pd.read_csv("3_siparis_pivot_data_canter_11_10.csv")
canter_sipariş

,yil,ay,malzeme_kodu,siparis_miktari
0,2021,1,0124525565,0.0
1,2021,2,0124525565,0.0
2,2021,3,0124525565,0.0
3,2021,4,0124525565,0.0
4,2021,5,0124525565,0.0
...,...,...,...,...
224365,2025,2,"['VL587189KP2', 'HD199107Y', 'ME263172AY']",10.0
224366,2025,3,"['VL587189KP2', 'HD199107Y', 'ME263172AY']",16.0
224367,2025,4,"['VL587189KP2', 'HD199107Y', 'ME263172AY']",2.0
224368,2025,5,"['VL587189KP2', 'HD199107Y', 'ME263172AY']",8.0


In [17]:
otob_sipariş=pd.read_csv("3_siparis_pivot_data_otobb_11_10.csv")
otob_sipariş

,yil,ay,malzeme_kodu,siparis_miktari
0,2021,1,0001231029,0.0
1,2021,2,0001231029,0.0
2,2021,3,0001231029,0.0
3,2021,4,0001231029,0.0
4,2021,5,0001231029,0.0
...,...,...,...,...
1249987,2025,2,"['ZF TE-ML 12', '0671-090-374']",0.0
1249988,2025,3,"['ZF TE-ML 12', '0671-090-374']",0.0
1249989,2025,4,"['ZF TE-ML 12', '0671-090-374']",0.0
1249990,2025,5,"['ZF TE-ML 12', '0671-090-374']",0.0


# Canter

In [18]:
canter_envanter=pd.read_csv("canter_envanter_11_10.csv")

In [19]:
import pandas as pd
df=canter_sipariş
# --- 1) Tarih oluştur ---
df["Tarih"] = pd.to_datetime(df["yil"].astype(str) + "-" + df["ay"].astype(str) + "-01")

ilk_tarih = df.groupby("malzeme_kodu")["Tarih"].min()
eligible_materials = ilk_tarih[ilk_tarih >= pd.Timestamp("2021-01-01")].index


mask = (
    (df["Tarih"] >= "2024-01-01") &
    (df["Tarih"] <= "2024-12-31") &
    (df["siparis_miktari"] > 0)
)
df_win = df.loc[mask & df["malzeme_kodu"].isin(eligible_materials)].copy()

df_win = df.loc[mask & df["malzeme_kodu"].isin(eligible_materials)].copy()

# --- 4) Her malzeme için benzersiz ay sayısı (frekans) ---
frekans = (
    df_win.groupby("malzeme_kodu")["Tarih"]
    .apply(lambda x: x.dt.to_period("M").nunique())
    .rename("Frekans")
)

# Pencerede hiç kaydı olmayan eligible malzemelere 0 ata
frekans_canter = frekans.reindex(eligible_materials, fill_value=0).reset_index()

# --- 5) Frekans kategorisi ---
def kategori(x: int) -> str:
    if x == 0:
        return "0"
    elif 1 <= x <= 2:
        return "1-2"
    elif 3 <= x <= 5:
        return "3-5"
    elif 6 <= x <= 8:
        return "6-8"
    elif 9 <= x <= 11:
        return "9-11"
    elif x == 12:
        return "12"
    else:
        return "Diğer"

frekans_canter["Frekans_Kategori"] = frekans_canter["Frekans"].apply(kategori)

frekans_canter.head()


,malzeme_kodu,Frekans,Frekans_Kategori
0,0124525565,3,3-5
1,026123001A,1,1-2
2,0281002778,2,1-2
3,0281004143,1,1-2
4,0281004209,4,3-5


In [20]:
frekans_canter = frekans_canter.rename(columns={"malzeme_kodu": "GuncelMalzeme"})

### Kritiklik

In [21]:
kritik = canter_envanter.groupby("GuncelMalzeme").agg({
    "Sipariş Kriteri": lambda x: any(x.isin(["K", "B", "R"]))
}).reset_index()

kritik.columns = ["GuncelMalzeme","Kritiklik_KBR"]


kategori_dataset = frekans_canter.merge(kritik, on="GuncelMalzeme", how="left")[
    ["GuncelMalzeme", "Frekans_Kategori","Kritiklik_KBR"]
]

### Kritikliği nan gelenler siparişde olup envanter verisinde olmayanlar

In [22]:
a = canter_envanter["GuncelMalzeme"].unique()
b = frekans_canter[~frekans_canter["GuncelMalzeme"].isin(a)]
d=b["GuncelMalzeme"].unique()
kategori_dataset[kategori_dataset["GuncelMalzeme"].isin(d)]

,GuncelMalzeme,Frekans_Kategori,Kritiklik_KBR
0,0124525565,3-5,NaN
1,026123001A,1-2,NaN
2,0281002778,1-2,NaN
3,0281004143,1-2,NaN
4,0281004209,3-5,NaN
...,...,...,...
466,MC118964Y,0,NaN
867,ME012908KP,0,NaN
908,ME018278KP,0,NaN
997,ME194174R,0,NaN


In [343]:
envanter_filtre = canter_envanter[["GuncelMalzeme", "VOR"]].copy()


envanter_filtre["VOR_Flag"] = envanter_filtre["VOR"].astype(str).str.upper().eq("X")
envanter_filtre = (
    envanter_filtre.groupby("GuncelMalzeme", as_index=False)["VOR_Flag"].any()
)


kategori_dataset = kategori_dataset.merge(
    envanter_filtre,
    on="GuncelMalzeme",
    how="left"
)


kategori_dataset["Kritiklik_VOR"] = kategori_dataset["VOR_Flag"]


kategori_dataset = kategori_dataset.drop(columns=["VOR_Flag"])

print(kategori_dataset.head())


  GuncelMalzeme Frekans_Kategori Kritiklik_KBR Kritiklik_VOR
0    0124525565              3-5           NaN           NaN
1    026123001A              1-2           NaN           NaN
2    0281002778              1-2           NaN           NaN
3    0281004143              1-2           NaN           NaN
4    0281004209              3-5           NaN           NaN


In [344]:
kategori_dataset["Kritiklik_KBR"].isnull().sum()        
      

np.int64(79)

In [345]:
kategori_dataset["Kritiklik_VOR"].isnull().sum()      

np.int64(79)

In [346]:
kategori_dataset["Kritiklik_KBR"].value_counts()

Kritiklik_KBR
False    3746
True      330
Name: count, dtype: int64

In [347]:
kategori_dataset["Kritiklik_VOR"].value_counts()

Kritiklik_VOR
False    3529
True      547
Name: count, dtype: int64

#### İki kritiğin birleşiminden kritiklik yaratma

In [348]:
c=kategori_dataset.copy()

In [349]:
c["Kritiklik_KBR"] = c["Kritiklik_KBR"].fillna(False)
c["Kritiklik_VOR"] = c["Kritiklik_VOR"].fillna(False)

C:\Users\Asus\AppData\Local\Temp\ipykernel_25448\1945816431.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  c["Kritiklik_KBR"] = c["Kritiklik_KBR"].fillna(False)
C:\Users\Asus\AppData\Local\Temp\ipykernel_25448\1945816431.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  c["Kritiklik_VOR"] = c["Kritiklik_VOR"].fillna(False)


In [350]:
c["Kritiklik"] = (
    c["Kritiklik_KBR"] | c["Kritiklik_VOR"]
)


In [351]:
c[["Kritiklik_KBR", "Kritiklik_VOR", "Kritiklik"]] = (
    c[["Kritiklik_KBR", "Kritiklik_VOR", "Kritiklik"]].astype(int)
)

In [353]:
c[c["Kritiklik"]==0][0:50]

,GuncelMalzeme,Frekans_Kategori,Kritiklik_KBR,Kritiklik_VOR,Kritiklik
0,0124525565,3-5,0,0,0
1,026123001A,1-2,0,0,0
2,0281002778,1-2,0,0,0
3,0281004143,1-2,0,0,0
4,0281004209,3-5,0,0,0
5,0281006032,0,0,0,0
6,0281006087,1-2,0,0,0
7,0281006154,9-11,0,0,0
8,0281006164,3-5,0,0,0
9,0444021014,1-2,0,0,0


### Son siparişten geçen süre

In [354]:
import pandas as pd


ref_date = pd.to_datetime("2024-12-30")
df=canter_sipariş[canter_sipariş["Tarih"]<"2025-01-01"]
df=df.rename(columns={"malzeme_kodu":"GuncelMalzeme"})
df=df[df["siparis_miktari"]>0]
last_order = (
    df.groupby("GuncelMalzeme")["Tarih"].max().reset_index()
)
last_order.columns = ["GuncelMalzeme", "SonSiparisTarihi"]


last_order["SonSiparistenGecenSüre_Ay"] = (
    (ref_date.year - last_order["SonSiparisTarihi"].dt.year) * 12
    + (ref_date.month - last_order["SonSiparisTarihi"].dt.month)
)

c = c.merge(
    last_order[["GuncelMalzeme", "SonSiparistenGecenSüre_Ay"]],
    on="GuncelMalzeme",
    how="left"
)



In [172]:
canter_sipariş[canter_sipariş["malzeme_kodu"]=="['VL587184KP2', 'HD199127Y']"]

,yil,ay,malzeme_kodu,siparis_miktari,Tarih
224208,2021,1,"['VL587184KP2', 'HD199127Y']",0.0,2021-01-01
224209,2021,2,"['VL587184KP2', 'HD199127Y']",0.0,2021-02-01
224210,2021,3,"['VL587184KP2', 'HD199127Y']",0.0,2021-03-01
224211,2021,4,"['VL587184KP2', 'HD199127Y']",0.0,2021-04-01
224212,2021,5,"['VL587184KP2', 'HD199127Y']",0.0,2021-05-01
224213,2021,6,"['VL587184KP2', 'HD199127Y']",0.0,2021-06-01
224214,2021,7,"['VL587184KP2', 'HD199127Y']",0.0,2021-07-01
224215,2021,8,"['VL587184KP2', 'HD199127Y']",0.0,2021-08-01
224216,2021,9,"['VL587184KP2', 'HD199127Y']",1.0,2021-09-01
224217,2021,10,"['VL587184KP2', 'HD199127Y']",0.0,2021-10-01


In [169]:
c

,GuncelMalzeme,Frekans_Kategori,Kritiklik_KBR,Kritiklik_VOR,Kritiklik,SonSiparistenGecenSüre_Ay
0,0124525565,3-5,0,0,0,5.0
1,026123001A,1-2,0,0,0,5.0
2,0281002778,1-2,0,0,0,2.0
3,0281004143,1-2,0,0,0,7.0
4,0281004209,3-5,0,0,0,3.0
...,...,...,...,...,...,...
4150,"['TM199100KP2', 'HD199120Y']",3-5,0,0,0,2.0
4151,"['TM302690Y', 'MB302690Y']",0,0,0,0,15.0
4152,"['VL587184KP2', 'HD199127Y']",0,0,0,0,26.0
4153,"['VL587188KP2', 'HD199126Y']",1-2,0,0,0,1.0


In [356]:
c.to_excel("Canter_kategori_2.xlsx")

# OTOBB

In [357]:
otob_envanter=pd.read_csv("otob_envanter_11_10.csv")

C:\Users\Asus\AppData\Local\Temp\ipykernel_25448\3345154246.py:1: DtypeWarning: Columns (3,4,33,36,45) have mixed types. Specify dtype option on import or set low_memory=False.
  otob_envanter=pd.read_csv("otob_envanter_11_10.csv")


### Frekans

In [358]:
import pandas as pd
df=otob_sipariş

df["Tarih"] = pd.to_datetime(df["yil"].astype(str) + "-" + df["ay"].astype(str) + "-01")

ilk_tarih = df.groupby("malzeme_kodu")["Tarih"].min()
eligible_materials = ilk_tarih[ilk_tarih >= pd.Timestamp("2021-01-01")].index


mask = (
    (df["Tarih"] >= "2024-01-01") &
    (df["Tarih"] <= "2024-12-31") &
    (df["siparis_miktari"] > 0)
)
df_win = df.loc[mask & df["malzeme_kodu"].isin(eligible_materials)].copy()

df_win = df.loc[mask & df["malzeme_kodu"].isin(eligible_materials)].copy()


frekans = (
    df_win.groupby("malzeme_kodu")["Tarih"]
    .apply(lambda x: x.dt.to_period("M").nunique())
    .rename("Frekans")
)


frekans_otob = frekans.reindex(eligible_materials, fill_value=0).reset_index()


def kategori(x: int) -> str:
    if x == 0:
        return "0"
    elif 1 <= x <= 2:
        return "1-2"
    elif 3 <= x <= 5:
        return "3-5"
    elif 6 <= x <= 8:
        return "6-8"
    elif 9 <= x <= 11:
        return "9-11"
    elif x == 12:
        return "12"
    else:
        return "Diğer"

frekans_otob["Frekans_Kategori"] = frekans_otob["Frekans"].apply(kategori)

frekans_otob.head()


,malzeme_kodu,Frekans,Frekans_Kategori
0,0001231029,0,0
1,0001231030,0,0
2,0001263006,1,1-2
3,0010123Y,0,0
4,0010302000,0,0


In [359]:
frekans_otob = frekans_otob.rename(columns={"malzeme_kodu": "GuncelMalzeme"})

### Kritiklik

In [360]:
kritik = otob_envanter.groupby("GuncelMalzeme").agg({
    "Sipariş Kriteri": lambda x: any(x.isin(["K", "B", "R"]))
}).reset_index()

kritik.columns = ["GuncelMalzeme","Kritiklik_KBR"]


kategori_dataset = frekans_otob.merge(kritik, on="GuncelMalzeme", how="left")[
    ["GuncelMalzeme", "Frekans_Kategori","Kritiklik_KBR"]
]

In [361]:
a=otob_envanter["GuncelMalzeme"].unique()
b = frekans_otob[~frekans_otob["GuncelMalzeme"].isin(a)]
d=b["GuncelMalzeme"].unique()
kategori_dataset[kategori_dataset["GuncelMalzeme"].isin(d)]

,GuncelMalzeme,Frekans_Kategori,Kritiklik_KBR
0,0001231029,0,NaN
1,0001231030,0,NaN
2,0001263006,1-2,NaN
3,0010123Y,0,NaN
4,0010302000,0,NaN
...,...,...,...
21054,"['0010112Y', 'TM212136-01Y']",6-8,NaN
21057,"['01573719Y', 'TM187142-02Y']",12,NaN
21058,"['0209-016810Y', 'TM240157-05Y']",9-11,NaN
21085,"['0501-325-886', '4181-298-002']",9-11,NaN


In [362]:
envanter_filtre = otob_envanter[["GuncelMalzeme", "VOR"]].copy()


envanter_filtre["VOR_Flag"] = envanter_filtre["VOR"].astype(str).str.upper().eq("X")
envanter_filtre = (
    envanter_filtre.groupby("GuncelMalzeme", as_index=False)["VOR_Flag"].any()
)


kategori_dataset = kategori_dataset.merge(
    envanter_filtre,
    on="GuncelMalzeme",
    how="left"
)


kategori_dataset["Kritiklik_VOR"] = kategori_dataset["VOR_Flag"]

kategori_dataset = kategori_dataset.drop(columns=["VOR_Flag"])

print(kategori_dataset.head())


  GuncelMalzeme Frekans_Kategori Kritiklik_KBR Kritiklik_VOR
0    0001231029                0           NaN           NaN
1    0001231030                0           NaN           NaN
2    0001263006              1-2           NaN           NaN
3      0010123Y                0           NaN           NaN
4    0010302000                0           NaN           NaN


In [363]:
kategori_dataset["Kritiklik_KBR"].isnull().sum()    

np.int64(369)

In [364]:
kategori_dataset["Kritiklik_VOR"].isnull().sum()    

np.int64(369)

In [365]:
kategori_dataset["Kritiklik_KBR"].value_counts()

Kritiklik_KBR
False    21136
True      1643
Name: count, dtype: int64

In [366]:
kategori_dataset["Kritiklik_VOR"].value_counts()

Kritiklik_VOR
False    21445
True      1334
Name: count, dtype: int64

### Kritiklik sutunu yaratma(vor kbr birleşimi)

In [367]:
o=kategori_dataset.copy()

In [368]:
o["Kritiklik_KBR"] = o["Kritiklik_KBR"].fillna(False)
o["Kritiklik_VOR"] = o["Kritiklik_VOR"].fillna(False)

C:\Users\Asus\AppData\Local\Temp\ipykernel_25448\2105829306.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  o["Kritiklik_KBR"] = o["Kritiklik_KBR"].fillna(False)
C:\Users\Asus\AppData\Local\Temp\ipykernel_25448\2105829306.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  o["Kritiklik_VOR"] = o["Kritiklik_VOR"].fillna(False)


In [369]:
o["Kritiklik"] = (
    o["Kritiklik_KBR"] | o["Kritiklik_VOR"]
)


In [370]:
o[["Kritiklik_KBR", "Kritiklik_VOR", "Kritiklik"]] = (
    o[["Kritiklik_KBR", "Kritiklik_VOR", "Kritiklik"]].astype(int)
)

In [371]:
o[o["Kritiklik"]==0].iloc[0:50]

,GuncelMalzeme,Frekans_Kategori,Kritiklik_KBR,Kritiklik_VOR,Kritiklik
0,0001231029,0,0,0,0
1,0001231030,0,0,0,0
2,0001263006,1-2,0,0,0
3,0010123Y,0,0,0,0
4,0010302000,0,0,0,0
5,00120X,9-11,0,0,0
6,0023336033,0,0,0,0
7,003 569-03,1-2,0,0,0
8,0066409049,0,0,0,0
9,006714Y,0,0,0,0


### Son siparişten geçen süre

In [372]:
import pandas as pd


ref_date = pd.to_datetime("2024-12-30")
df=otob_sipariş[otob_sipariş["Tarih"]<"2025-01-01"]
df=df.rename(columns={"malzeme_kodu":"GuncelMalzeme"})
df=df[df["siparis_miktari"]>0]
last_order = (
    df.groupby("GuncelMalzeme")["Tarih"].max().reset_index()
)
last_order.columns = ["GuncelMalzeme", "SonSiparisTarihi"]

last_order["SonSiparistenGecenSüre_Ay"] = (
    (ref_date.year - last_order["SonSiparisTarihi"].dt.year) * 12
    + (ref_date.month - last_order["SonSiparisTarihi"].dt.month)
)

o = o.merge(
    last_order[["GuncelMalzeme", "SonSiparistenGecenSüre_Ay"]],
    on="GuncelMalzeme",
    how="left"
)


### 2025 öncesi hiç siparişi yoksa

In [384]:
otob_sipariş[otob_sipariş["malzeme_kodu"]=="['PB100011Y', 'TU252147Y']"]

,yil,ay,malzeme_kodu,siparis_miktari,Tarih
1231956,2021,1,"['PB100011Y', 'TU252147Y']",0.0,2021-01-01
1231957,2021,2,"['PB100011Y', 'TU252147Y']",0.0,2021-02-01
1231958,2021,3,"['PB100011Y', 'TU252147Y']",0.0,2021-03-01
1231959,2021,4,"['PB100011Y', 'TU252147Y']",0.0,2021-04-01
1231960,2021,5,"['PB100011Y', 'TU252147Y']",0.0,2021-05-01
1231961,2021,6,"['PB100011Y', 'TU252147Y']",0.0,2021-06-01
1231962,2021,7,"['PB100011Y', 'TU252147Y']",0.0,2021-07-01
1231963,2021,8,"['PB100011Y', 'TU252147Y']",0.0,2021-08-01
1231964,2021,9,"['PB100011Y', 'TU252147Y']",0.0,2021-09-01
1231965,2021,10,"['PB100011Y', 'TU252147Y']",0.0,2021-10-01


In [377]:
o[o["SonSiparistenGecenSüre_Ay"].isnull()]

,GuncelMalzeme,Frekans_Kategori,Kritiklik_KBR,Kritiklik_VOR,Kritiklik,SonSiparistenGecenSüre_Ay
19,0178-XB-03-08-05_A,0,0,0,0,NaN
23,0178-XD-01-06-02_A,0,0,0,0,NaN
25,0178-XD-01-06-05_A,0,0,0,0,NaN
27,0178-XD-01-08-02_A,0,0,0,0,NaN
53,02.89.00.528Y,0,0,0,0,NaN
...,...,...,...,...,...,...
22814,"['PB100011Y', 'TU252147Y']",0,0,0,0,NaN
22829,"['QC000055', 'MK667569']",0,0,0,0,NaN
22863,"['RP150403', 'RP150402']",0,0,0,0,NaN
23139,"['USB TO 2CH RS485', '8160-196Y']",0,0,0,0,NaN


In [94]:
o.to_excel("OTOBB_categori_2.xlsx")

### HATA

In [26]:
excel_path = r"Partvision Envanter 14.08.25 1 (4).xlsx"
envanter = pd.read_excel(excel_path, sheet_name="4000 canter")

In [27]:
kategori_dataset["Kritiklik_KBR"].isnull()  

0        True
1        True
2        True
3        True
4        True
        ...  
4150    False
4151    False
4152    False
4153    False
4154    False
Name: Kritiklik_KBR, Length: 4155, dtype: bool

In [28]:
kategori_dataset

,GuncelMalzeme,Frekans_Kategori,Kritiklik_KBR
0,0124525565,3-5,NaN
1,026123001A,1-2,NaN
2,0281002778,1-2,NaN
3,0281004143,1-2,NaN
4,0281004209,3-5,NaN
...,...,...,...
4150,"['TM199100KP2', 'HD199120Y']",3-5,False
4151,"['TM302690Y', 'MB302690Y']",0,False
4152,"['VL587184KP2', 'HD199127Y']",0,False
4153,"['VL587188KP2', 'HD199126Y']",1-2,False


In [29]:
kbr=kategori_dataset[kategori_dataset["Kritiklik_KBR"].isnull()]    

In [32]:
kbr["GuncelMalzeme"].unique()

array(['0124525565', '026123001A', '0281002778', '0281004143',
       '0281004209', '0281006032', '0281006087', '0281006154',
       '0281006164', '0444021014', '0445010559', '0445110564',
       '0445224036', '0445224055', '1010-42773-01Y', '1010-42773-02Y',
       '1010-42773Y', '1010-43014Y', '1052A192', '1052A193', '1052B175',
       '1110010013', '1122802600', '1250A015', '1258A002', '1465ZS0069',
       '1500-32431', '1511-42880Y', '1570-42389', '1609909', '1700-42028',
       '200293INY', '2031-KAN-01Y', '2171.01000025', '2200-32761',
       '2211-48840Y', '2211-48841Y', '2996623', '2996842', '3063-33109Y',
       '3124054500', '316BC2KVMY', '3242-KAN-02Y', '42576090',
       '4331-KAN-01Y', '4331-KAN-02Y', '500054911', '500086201Y',
       '500086207', '500301568', '5004-KAN-01Y', '5004-KAN-02Y',
       '504027865', '504074159', '504081697', '504243641', '504317810',
       '504320901', '504325903TPS', '504329658', '504334709', '504342424',
       '504351882', '504353905', '504

In [281]:
vor=kategori_dataset[kategori_dataset["Kritiklik_VOR"].isnull()]    

In [283]:
vor=vor["GuncelMalzeme"].unique()

In [191]:
envanter


,Malzeme Kodu,Malzeme Tanımı,Değişen Malzeme Kodu,Yerli Malzeme Kodu,Muadil Malzeme Kodu,Toplam Stok Miktarı,Kullanılabilir stok adedi,Back order adedi,"Açık, stok sip.adt.","Açık, acil sip.adt.",...,Yurtiçi Rabat,Yurtdışı Rabat,Malzeme Tanımı ( EN ),Üretim Yeri,Key Cluster Kodu,Üst Sınıf Kodu,Ana Grup Kodu,VOR,Teps,Phase Out
0,0123525501,ALTERNATÖR 100 A,0123525501Y,0123525501Y,NaN,0,0,0,0,0,...,7.0,D,"ALTERNATOR, 100 A",4000,Y0010,U001,G01,NaN,X,NaN
1,0123525501Y,ALTERNATOR 100 A,0123525507,NaN,NaN,0,0,0,0,0,...,5.0,C,ALTERNATOR 100 A,4000,Y0010,U001,G01,NaN,X,NaN
2,0123525507,ALTERNATOR 100 A,NaN,NaN,NaN,0,0,0,0,0,...,6.0,C,NaN,4000,Y0010,U001,G01,NaN,X,NaN
3,0124525565,ALTERNATOR,NaN,NaN,NaN,0,0,0,1,0,...,6.0,C,NaN,4000,Y0010,U001,G01,X,X,NaN
4,0178-XC-11-14-01_A,DIN935 M14X2 A2 PASLANMAZ YARIKLI SOMUN,NaN,NaN,NaN,0,0,0,0,0,...,NaN,L,DIN935 M14X2 A2 STAINLESS SLOT NUT,4000,Y0333,U112,G27,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15384,YPTAKOZ1,TAHTA TAKOZ 5X10X300 CM,NaN,NaN,NaN,0,0,0,0,0,...,NaN,L,WOOD WEDGE 5X10X300 CM,4000,Y0446,U122,G27,NaN,NaN,NaN
15385,YPTAKOZ2,TAKOZ 5X5X300,NaN,NaN,NaN,0,0,0,0,0,...,NaN,NaN,NaN,4000,Y0355,U122,G27,NaN,NaN,NaN
15386,ZA397924,"GEARASSY, MAINSHAFT 2ND",NaN,NaN,NaN,0,0,0,0,0,...,5.0,NaN,"GEARASSY, MAINSHAFT 2ND",4000,Y0090,U007,G02,NaN,NaN,NaN
15387,ZA397925,"GEARASSY, MAINSHAFT 3ND",NaN,NaN,NaN,0,0,0,0,0,...,5.0,NaN,"GEARASSY, MAINSHAFT 3ND",4000,Y0090,U007,G02,NaN,NaN,NaN


In [287]:
olmayan_envanter=envanter[envanter["Malzeme Kodu"].isin(vor)]

In [290]:
olmayan_envanter

,Malzeme Kodu,Malzeme Tanımı,Değişen Malzeme Kodu,Yerli Malzeme Kodu,Muadil Malzeme Kodu,Toplam Stok Miktarı,Kullanılabilir stok adedi,Back order adedi,"Açık, stok sip.adt.","Açık, acil sip.adt.",Tedarik türü,Tedarik yeri,Araç tipi,Standart maliyet,Ortalama maliyet,Yeni Sprş Seviyesi,Azami Stok,Mip Parti Byklğ,Mip Karakteristiği,Plnlı Tslmat Srsi,Sabitleme Srsi,Lead Time,Özel Tdrk,Blokaj,Parça Açıklaması,Replacibility Kod,Değişen Parça Replacibility,Orjinal Parca no,Orj. Parca Stok Ad.,Sipariş Kriteri,Cluster,Üst Sınıf,Ana Grup,Min. Order Qty.,Min. Package Qty.,Mevsimsel,Yurtiçi Rabat,Yurtdışı Rabat,Malzeme Tanımı ( EN ),Üretim Yeri,Key Cluster Kodu,Üst Sınıf Kodu,Ana Grup Kodu,VOR,Teps,Phase Out
3,0124525565,ALTERNATOR,NaN,NaN,NaN,0,0,0,1,0,Y,BOSCH-TRY,DIGER,8616.89,7290.58,0,0,42,PD,60,NaN,63,NaN,NaN,"* Otobüste Yaz dönemi sıkıntı yaşanan parçadır, dikkat!!",O,NaN,NaN,0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,6.0,C,NaN,4000,Y0010,U001,G01,X,X,NaN
5,026123001A,EMME MANIFOLDU HAVA SENSORU,NaN,NaN,NaN,4,4,0,0,0,I,BOSCH-TRY,CANTERORT,559.28,559.28,0,0,42,PD,60,NaN,63,NaN,NaN,NaN,O,NaN,NaN,0,NaN,EMME MANİFOLTU,MOTOR DETAYLARI,MOTOR,NaN,NaN,NaN,5.0,C,INTAKE MANIFOLD AIR SENSOR,4000,Y0102,U001,G01,NaN,NaN,NaN
8,0281002778,ELECTRON.TRANSMITT,NaN,NaN,NaN,4,4,0,0,0,I,BOSCH-TRY,PRESTIJ E6,512.36,401.49,0,0,42,PD,60,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0,H,SENSÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,6.0,C,"SENSOR,ENG SPEED",4000,Y0323,U001,G01,NaN,NaN,NaN
12,0281004143,"SENSOR,EGSOZ MONTAJI",NaN,0281004143Y,NaN,0,0,1,0,0,I,BOSCH-TRY,PRESTIJ E6,1339.11,1519.71,0,0,42,PD,75,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0,H,SENSÖR,EGSOZ SUSTURUCU VE DETAYLARI,EGSOZ SİSTEMİ,NaN,NaN,M2,6.0,C,"SENSOR,EXHAUST ASSY",4000,Y0323,U028,G08,NaN,NaN,NaN
14,0281004209,SENSÖR,NaN,0281004209Y,NaN,3,3,0,0,0,Y,BOSCH-TRY,PRESTIJ E6,1304.18,2298.23,0,0,42,PD,75,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0,H,SENSÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,6.0,C,NaN,4000,Y0323,U001,G01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,504334709,"E5-E6 PRESTIJ GERGI RULMANI,CIFTLI",NaN,NaN,NaN,0,0,0,190,0,I,FPT,PRESTIJ E6,1396.40,1143.81,0,0,42,PD,60,NaN,63,NaN,NaN,NaN,NaN,NaN,MK667237Y,7,H,RULMAN,FAN TAHRİK DETAYLARI,MOTOR,NaN,NaN,M2,7.0,B,"E5-E6 PRESTIJ TENSIONER BEARING,DOUBLE",4000,Y0315,U004,G01,NaN,X,NaN
400,504342424,"RAY KOMPLESİ,YAKIT,MOTOR",NaN,NaN,NaN,0,0,3,9,0,I,FPT,FEB91E,6390.01,6179.72,0,0,42,PD,75,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0,M,RAY,YAKIT HATTI DETAYLARI,MOTOR,NaN,NaN,M1,4.0,B,"RAIL COMPLETE,FUEL,ENGINE",4000,Y0305,U023,G01,NaN,X,NaN
403,504351882,"CONTA,EGR BORUSU",NaN,NaN,NaN,10,10,0,9,0,I,FPT,CANTER S-E,85.24,73.87,0,0,42,PD,75,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0,O,CONTA,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,4.0,B,"GASKET,EGR PIPE",4000,Y0063,U001,G01,NaN,X,NaN
404,504353905,TERMOSTAT,NaN,QC000106AG,NaN,40,40,0,498,0,I,FPT,CANTER S-E,780.18,726.44,17,28,HB,V1,75,NaN,63,NaN,NaN,NaN,O,NaN,NaN,0,H,TERMOSTAD,MOTOR DETAYLARI,MOTOR,NaN,NaN,M1,7.0,B,THERMOSTAT,4000,Y0366,U001,G01,X,X,NaN


In [294]:
olmayan_envanter["Değişen Malzeme Kodu"].value_counts(dropna=False)


Değişen Malzeme Kodu
NaN    65
Name: count, dtype: int64

In [295]:
olmayan_envanter["Muadil Malzeme Kodu"].value_counts(dropna=False)

Muadil Malzeme Kodu
NaN    65
Name: count, dtype: int64

In [297]:
olmayan_envanter["Sabitleme Srsi"].value_counts(dropna=False)

Sabitleme Srsi
NaN    65
Name: count, dtype: int64

In [300]:
olmayan_envanter["Blokaj"].value_counts(dropna=False)

Blokaj
NaN    61
CA      2
TF      2
Name: count, dtype: int64

In [ ]:
Phase Out

In [301]:
olmayan_envanter["Değişen Parça Replacibility"].value_counts(dropna=False)

Değişen Parça Replacibility
NaN    65
Name: count, dtype: int64

In [302]:
olmayan_envanter["Min. Order Qty."].value_counts(dropna=False)

Min. Order Qty.
NaN    64
5.0     1
Name: count, dtype: int64

In [303]:
olmayan_envanter["Phase Out"].value_counts(dropna=False)

Phase Out
NaN    65
Name: count, dtype: int64